In [44]:
import csv
capacidad_x_centro_ruta = 'capacidades_reducido.csv'
distancia_vot_centro_ruta = 'constantes_reducido.csv'
CAP_MIN = 30

In [45]:
#Creacion del diccionario capacidad_x_centro = {id_centro: capacidad}

capacidad_x_centro = {}
with open(capacidad_x_centro_ruta, "r") as f:
    capacidad_x_centro_csv = csv.reader(f)
    principio = True
    for id_centro, capacidad in capacidad_x_centro_csv:
        if principio:
            principio = False
            continue
        capacidad_x_centro[int(id_centro)] = int(capacidad)

cant_centros = len(capacidad_x_centro)

In [46]:
#Creacion del diccionario distancia_vot_centro = {(id_votante, id_centro): distancia}

distancia_vot_centro = {}

with open(distancia_vot_centro_ruta, "r") as f:
    distancia_vot_centro_csv = csv.reader(f)
    principio = True
    for id_votante, id_centro, distancia in distancia_vot_centro_csv:
        if principio:
            principio = False
            continue
        distancia_vot_centro[(int(id_votante), int(id_centro))] = float(distancia)

In [47]:
def fairness(distancia):
    return distancia**2

In [48]:
#Creacion del diccionario distancia_a_centros = {id_votante: {id_centro: distancia}}

cant_votantes = int(len(distancia_vot_centro.keys())/len(capacidad_x_centro.keys()))

distancia_a_centros = {}

for id_votante in range(1,cant_votantes+1):
    distancia_por_centro = {}
    for centro in capacidad_x_centro:
        distancia_por_centro[centro] = distancia_vot_centro[(id_votante, centro)]
    distancia_a_centros[id_votante] = distancia_por_centro

In [49]:
#Creacion del diccionario fairness_a_centros = {id_votante: {id_centro: distancia_con_fairness}}

fairness_a_centros = {}

for id_votante in range(1,cant_votantes+1):
    fairness_por_centro = {}
    for centro in capacidad_x_centro:
        fairness_por_centro[centro] = fairness(distancia_vot_centro[(id_votante, centro)])
    fairness_a_centros[id_votante] = fairness_por_centro

In [50]:
def regla(distancia, fairness):
    return distancia + fairness

In [51]:
#Creacion del diccionario reglas = {id_votante: {id_centro: distancia_con_fairness + distancia}}

reglas = {}

for id_votante in range(1, cant_votantes+1):
    reglas_para_votante = {}
    for id_centro in range(1, cant_centros+1):
        reglas_para_votante[id_centro] = regla(distancia_a_centros[id_votante][id_centro], fairness_a_centros[id_votante][id_centro])
    reglas[id_votante] = reglas_para_votante

In [52]:
#Primera corrida -> asignacion de todos los votantes al centro mas cercano

asignados = {}

for id_votante in range(1, cant_votantes+1):
        centros_ordenados = sorted(reglas[id_votante].items(), key=lambda kv: kv[1])
        for id_centro, distancia in centros_ordenados:
            cap_max = capacidad_x_centro[id_centro]
            asignacion = asignados.get(id_centro, [])
            if len(asignacion) < cap_max:
                asignacion.append(id_votante)
                asignados[id_centro] = asignacion
                break

In [53]:
#Verificacion de que esten todos los votantes asignados

cant_votantes_asignados = 0 
for asignacion in asignados:
    cant_votantes_asignados += len(asignados[asignacion])

if cant_votantes_asignados == cant_votantes:
    print("Estan todos los votantes asignados")
else:
    print("No estan todos los votantes asignados. Asignados: {} - cant_votantes: {}".format(cant_votantes_asignados, cant_votantes))

Estan todos los votantes asignados


In [54]:
#Chequeo que no haya centros con mas de su capacidad maxima

for id_centro in asignados:
    cap_max = capacidad_x_centro[id_centro]
    if len(asignados[id_centro]) > cap_max:
        print("Error en centro {} - cantidad {} - cap_max {}".format(id_centro, len(asignados[id_centro]), cap_max))

In [55]:
#Chequeo si hay centros con menos de la capacidad minima

for id_centro in asignados:
    if len(asignados[id_centro]) < CAP_MIN:
        print("Centro inhabilitado {} - cantidad {}".format(id_centro, len(asignados[id_centro])))

Centro inhabilitado 18 - cantidad 20
Centro inhabilitado 25 - cantidad 22
Centro inhabilitado 27 - cantidad 23
Centro inhabilitado 2 - cantidad 17
Centro inhabilitado 12 - cantidad 18
Centro inhabilitado 22 - cantidad 28
Centro inhabilitado 19 - cantidad 15
Centro inhabilitado 11 - cantidad 15
Centro inhabilitado 13 - cantidad 4
Centro inhabilitado 10 - cantidad 17
Centro inhabilitado 29 - cantidad 13
Centro inhabilitado 17 - cantidad 11
Centro inhabilitado 21 - cantidad 9
Centro inhabilitado 30 - cantidad 12
Centro inhabilitado 5 - cantidad 5
Centro inhabilitado 26 - cantidad 5
Centro inhabilitado 23 - cantidad 2
Centro inhabilitado 24 - cantidad 13
Centro inhabilitado 6 - cantidad 12


In [56]:
#Redistribucion del minimo 

for id_centro_a_reasignar in asignados:
    if len(asignados[id_centro_a_reasignar]) < CAP_MIN:
        for id_votante in asignados[id_centro_a_reasignar]:
            centros_ordenados = sorted(reglas[id_votante].items(), key=lambda kv: kv[1])
            for id_centro_posible, distancia in centros_ordenados:
                cap_max = capacidad_x_centro[id_centro_posible]
                if id_centro_posible in asignados and len(asignados[id_centro_posible]) > CAP_MIN and len(asignados[id_centro_posible]) < cap_max:
                    asignados[id_centro_posible].append(id_votante)
                    break
        asignados[id_centro_a_reasignar] = []

# Asignacion de los votantes al centro mas cercano, si no entra, al que sigue
# Algunos centros quedan con capacidad menor a la minima, pero nunca mayor a la maxima por la condicion anterior
# Redistribucion de los que tienen capacidad menor a la minima a los mas cercanos que no esten llenos y tengan capacidad mayor a la minima

In [57]:
#Chequeo que no haya centros con menos de su capacidad minima

for id_centro in asignados:
    if len(asignados[id_centro]) < CAP_MIN and len(asignados[id_centro]) > 0:
        print("Se asigno mal el centro {} con la cantidad {}".format(id_centro, len(asignados[id_centro])))
        
#Chequeo que no haya centros con mas de su capacidad maxima

for id_centro in asignados:
    cap_max = capacidad_x_centro[id_centro]
    if len(asignados[id_centro]) > cap_max:
        print("Error en centro {} - cantidad {} - cap_max {}".format(id_centro, len(asignados[id_centro]), cap_max))

In [58]:
#Verificacion de que esten todos los votantes asignados

cant_votantes_asignados = 0 
for asignacion in asignados:
    cant_votantes_asignados += len(asignados[asignacion])

if cant_votantes_asignados == cant_votantes:
    print("Estan todos los votantes asignados")
else:
    print("No estan todos los votantes asignados. Asignados: {} - cant_votantes: {}".format(cant_votantes_asignados, cant_votantes))

Estan todos los votantes asignados


In [59]:
resultado_ruta = "bivalente_encendidas_reducido_heuristica.csv"

with open(resultado_ruta, 'w', newline='') as csvfile:
    fieldnames = ['id_votante', 'id_centro']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for id_centro in asignados:
        for id_votante in asignados[id_centro]:
            writer.writerow({'id_votante': id_votante, 'id_centro': id_centro})